In [3]:
import os
import time

import pandas as pd
import polars as pl
import yfinance as yf
from dotenv import load_dotenv
from investment_info_fetcher import investment_info_fetcher

# PostgreSQLへの接続情報
load_dotenv()
DB_HOST: str | None = os.getenv("DB_HOST")
DB_PORT: str | None = os.getenv("DB_PORT")
DB_USER: str | None = os.getenv("DB_USER")
DB_PASSWORD: str | None = os.getenv("DB_PASSWORD")
DB_NAME: str | None = os.getenv("DB_NAME")

In [4]:
df_investment_info: pl.DataFrame = investment_info_fetcher(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)

In [47]:
yf_tickers = yf.Tickers(tickers=['VTI', 'VGK'])
df_yf = yf_tickers.history(period='1y', interval='1d')['Dividends']

df_yf = (
    pl.DataFrame(df_yf.reset_index())
    .rename({"Date": "date"})
    .with_columns(pl.col("date").cast(pl.Date))
)

yf_ticker = yf.Ticker('JPY=X')
df_yf_jpy = yf_ticker.history(period='1y', interval='1d')['Close']

df_yf_jpy = (
    pl.DataFrame(df_yf_jpy.reset_index())
    .rename({"Date": "date"})
    .with_columns(pl.col("date").cast(pl.Date))
)

df = df_yf.join(df_yf_jpy, on='date', how='inner')
df.filter(pl.col('VGK') > 0)

[*********************100%***********************]  2 of 2 completed


date,VGK,VTI,Close
date,f64,f64,f64
2024-03-15,0.37,0.0,148.296005
2024-12-20,0.719,0.0,157.643997
